In [1]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
# [CLS] -> BertModel -> Linear -> tanh -> Linear
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [4]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
bert_sc = bert_sc.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers.tokenization_utils_base import BatchEncoding


text_list = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。",
]
label_list = [
    1,
    0,
    1
]

# 符号化
encoding: BatchEncoding = tokenizer(
    text_list,
    padding="longest",
    return_tensors="pt",
)
encoding = {k: v.cuda() for k, v in encoding.items()}
labels = torch.tensor(label_list).cuda()

# 推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
# 分類スコア
# 形状は(バッチサイズ（文章数）、 カテゴリ数)
scores = output.logits
labels_predicted = scores.argmax(-1)

In [6]:
labels_predicted

tensor([1, 1, 1], device='cuda:0')

In [7]:
# モデルの分類器のパラメータ初期値はランダムな値のため、精度は低い
num_correct = (labels_predicted == labels).sum().item()
accuracy = num_correct / labels.size(0)
print(accuracy)

0.6666666666666666


In [8]:
# 符号化
encoding: BatchEncoding = tokenizer(
    text_list,
    padding="longest",
    return_tensors="pt",
)
encoding["labels"] = torch.tensor(label_list)
encoding = {k: v.cuda() for k, v in encoding.items()}

# ロスの計算
output = bert_sc(**encoding)
loss = output.loss
print(loss)

tensor(0.6297, device='cuda:0', grad_fn=<NllLossBackward0>)


OSコマンドで livedoor ニュースコーパスをダウンロード後、以下の処理につづく

## 6-5 BERT のファインチューニングと性能評価
- データ(符号化された文章)とラベルを抜き出し、ミニバッチにする
- データローダはデータセットからミニバッチを取り出す

In [10]:
dataset_for_loader = [
    {
        "data": torch.tensor([0, 1]),
        "labels": torch.tensor(0),
    },
    {
        "data": torch.tensor([2, 3]),
        "labels": torch.tensor(1),
    },
    {
        "data": torch.tensor([4, 5]),
        "labels": torch.tensor(2),
    },
    {
        "data": torch.tensor([6, 7]),
        "labels": torch.tensor(3),
    },
]
loader = DataLoader(dataset_for_loader, batch_size=2)

# ミニバッチを取り出す
for idx, batch in enumerate(loader):
    print(f"batch {idx}")
    print(batch)
    # ファインチューニングではここでミニバッチ毎の処理をおこなう

batch 0
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}
batch 1
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}


In [11]:
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)
for idx, batch in enumerate(loader):
    print(f"batch {idx}")
    print(batch)

batch 0
{'data': tensor([[4, 5],
        [2, 3]]), 'labels': tensor([2, 1])}
batch 1
{'data': tensor([[0, 1],
        [6, 7]]), 'labels': tensor([0, 3])}


### 前処理
各データを次のキーを持つ辞書にする
すなわち、 tokenizer で符号化を行ったときに得られる形式

- input_ids
- attention_mask
- token_type_ids
- labels

In [13]:
category_list = [
    "dokujo-tsushin",
    "it-life-hack",
    "kaden-channel",
    "livedoor-homme",
    "movie-enter",
    "peachy",
    "smax",
    "sports-watch",
    "topic-news",
]

tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# データ整形
# 学習の高速化のため128とする
max_length = 128
dataset_for_loader = []

for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f"./text/{category}/{category}*"):
        lines = open(file, encoding="utf-8").read().splitlines()
        # 4行目からが本文
        text = "\n".join(lines[3:])
        encoding = tokenizer(text, max_length=max_length, padding="max_length", truncation=True)
        encoding["labels"] = label
        # なぜこの場合はここでtensor化するのか? return_tensor ではだめなのか?
        encoding = {k: torch.tensor(v) for k, v in encoding.items()}
        dataset_for_loader.append(encoding)

100%|██████████| 9/9 [00:12<00:00,  1.38s/it]


In [15]:
from pprint import pprint
pprint(dataset_for_loader[0])
print(len(dataset_for_loader))

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]),
 'input_ids': tensor([    2,  2340, 19693, 10585, 28459,    35,  6692, 28493,    13,   501,
           62,   101,    37,     8,   569,   335,     5,    51,     7,     9,
         1040,     5,   616,     9,  2941,    18,  5602,   501,    20,    16,
         4027, 10531,   140,    36,    73, 30020, 28457, 25127,    38,  1080,
            5,    53,    28,   707,     5,    12,     9,    80,  3635,   205,
           29,  2935,   604,  5846,  6503,    11,  4722,    16,   861,    13,
            6, 12272, 24050,  2079,    11,    26,    62,    45,  

In [ ]:
# データセット分割
# 学習/検証/テスト = 6:2:2 で分割
random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)

dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train: n_train + n_val]
dataset_test = dataset_for_loader[n_train + n_val:]

# データセットからデータローダを作成
# 学習はshuffle=True
dataloader_train = DataLoader(
    dataset_train,
    # BERTのオリジナルの論文を参考に32としているとのこと
    batch_size=32,
    shuffle=True
)
dataloader_val = DataLoader(
    dataset_val,
    # 損失の勾配を計算しないので、大きめのバッチサイズ
    batch_size=256,
)
dataloader_test = DataLoader(
    dataset_test,
    batch_size=256,
)